In [1]:
import pandas as pd
import numpy as np
from scipy.spatial import distance
import math
# import pixiedust                   ##debuger
# import glob, os

In [2]:
all_stim = pd.read_csv('2018_07_05_21_18_19_stim.csv', sep=';')

In [3]:
responses = pd.read_csv('2018_07_05_21_18_19.csv', sep=';', encoding = "ISO-8859-1")

In [4]:
def area(x1, y1, x2, y2, x3, y3): 
    
    return abs((x1 * (y2 - y3) + 
                x2 * (y3 - y1) + 
                x3 * (y1 - y2)) / 2.0)

In [5]:
def check_area(x1, y1, x2, y2, lst1, lst2):
    r = 15
    e = -1
    #define lines coefficients
    m = (y1 - y2)/(x1-x2)
    b = y1 - m*x1
    if m != 0:
        m_c = e/m
    else:
        m_c = 0
    b1 = y1 - m_c*x1
    b2 = y2 - m_c*x2
    #define check points
    R = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)
    kx = (r/R) * (y2 - y1)
    ky = (r/R) * (x2 - x1)
    x1_check1 = x1 - kx
    y1_check1 = y1 + ky
    x1_check2 = x1 + kx
    y1_check2 = y1 - ky
    x2_check1 = x2 - kx
    y2_check1 = y2 + ky
    x2_check2 = x2 + kx
    y2_check2 = y2 - ky
    #check if the point inside the rectangle
    final_list= []
    for i in range(0, len(lst1)):#area of triangles and rectangle
        s1 = area(lst1[i], lst2[i], x1_check1, y1_check1, x1_check2, y1_check2)
        s2 = area(lst1[i], lst2[i], x1_check2, y1_check2, x2_check2, y2_check2)
        s3 = area(lst1[i], lst2[i], x2_check2, y2_check2, x2_check1, y2_check1)
        s4 = area(lst1[i], lst2[i], x2_check1, y2_check1, x1_check1, y1_check1)
        s_tr = round((s1 + s2 + s3 + s4), 3)
        s_rec = round((r * 2 * R), 3)
        #final check
        if s_tr == s_rec:
            final_list.append(1)
        else:
            final_list.append(0)
    return final_list  
            

# Feature task

In [6]:
#clean feature response with "bad response"
resp_feat = responses[responses["FeatConj"] == 0]
bad_feat = resp_feat.loc[(resp_feat['stType'] == ' distractor2 ') | (resp_feat['stType'] == ' distractor1 ') | (resp_feat['stType'] == 'distractor2') | (resp_feat['stType'] == 'distractor1'), 
                         'trialN'].tolist()
resp_feat = resp_feat[~resp_feat['trialN']
          .isin(bad_feat)]
resp_feat = resp_feat.reset_index(drop=True)
resp_feat.head()

,participantN,age,gender,trialN,task,condition,FeatConj,setSize,proportion,timelimit,...,col,row,touchX,touchY,touchDist,targDist,touchN,error,switches,changeScreen
0,133,0,NaN,0,0,r/g,0,80,50,0.0,...,0.0,0.0,122.5,117.0,0.0,0.000000,1,0,2,0
1,133,0,NaN,0,0,r/g,0,80,50,0.0,...,3.0,0.0,387.0,96.5,269.854127261378,272.261999,2,0,0,0
2,133,0,NaN,0,0,r/g,0,80,50,0.0,...,1.0,0.0,213.0,129.5,176.703848288598,165.595778,3,0,0,0
3,133,0,NaN,0,0,r/g,0,80,50,0.0,...,4.0,1.0,477.0,192.0,269.427634068965,245.669192,4,0,0,0
4,133,0,NaN,0,0,r/g,0,80,50,0.0,...,5.0,1.0,559.5,181.0,141.616559766152,75.368147,7,0,1,0


In [7]:
resp_feat['stType']

0      target1
1      target1
2      target1
3      target1
4      target2
        ...   
395    target2
396    target2
397    target2
398    target1
399    target1
Name: stType, Length: 400, dtype: object

In [8]:
all_stim.columns

Index(['participantN', 'age', 'gender', 'trialN', 'task', 'condition',
       'FeatConj', 'setSize', 'proportion', 'timelimit', 'col', 'row',
       'imgName', 'stType', 'posX', 'posY'],
      dtype='object')

In [9]:
resp_feat1 = resp_feat.rename(columns={'stType': 'type', 
                                          'stPosX': 'pos_x_r', 
                                          'stPosY': 'pos_y_r',
                                         'trialN': 'trial_r',
                                      'participantN': 'p_n'})
resp_feat1['pos_x_r'] = np.around(resp_feat1['pos_x_r'], decimals=2)
resp_feat1['pos_y_r'] = np.around(resp_feat1['pos_y_r'], decimals=2)
resp_feat['resp_pos'] = resp_feat1.apply(lambda x: list([x['pos_x_r'], x['pos_y_r']]) , axis=1)
resp_feat1['pos_r'] = list(zip(resp_feat1['pos_x_r'], resp_feat1['pos_y_r']))

resp_feat_c = resp_feat1.loc[:, resp_feat1.columns.intersection(['type', 
                                                             'pos_r',
                                                             'trial_r',
                                                               'p_n',
                                                                'pos_x_r',
                                                                'pos_y_r'])]

resp_feat1 = resp_feat1.loc[:, resp_feat1.columns.intersection(['type', 
                                                             'pos_r',
                                                             'trial_r',
                                                               'p_n'])]

resp_feat1 = resp_feat1.reset_index(drop=True)
resp_feat1.type = resp_feat1.type.str.strip()

### Add x1,x2,y1,y2 (I was too sleepy...)

In [10]:
resp_feat_c['x1'] = resp_feat_c['pos_x_r'].apply(lambda x: x + 108.6)
resp_feat_c['x2'] = resp_feat_c['pos_x_r'].apply(lambda x: x - 108.6)
resp_feat_c['y1'] = resp_feat_c['pos_y_r'].apply(lambda x: x + 106.5)
resp_feat_c['y2'] = resp_feat_c['pos_y_r'].apply(lambda x: x - 106.5)

### Distances

In [11]:
feat_r = resp_feat1.values.tolist()

In [12]:
#only useful df of all stimuli for feature task
all_feat = all_stim[all_stim["FeatConj"] == 0]
feat_target = all_feat[(all_feat['stType'] == 'target1') 
                       | (all_feat['stType'] == 'target2')]
feat_distr = all_feat[(all_feat['stType'] == 'distractor1') 
                      | (all_feat['stType'] == 'distractor2')]

In [13]:
#useful targets for feature (only 3 columns)
feat_rest_t = feat_target[~feat_target['trialN']
          .isin(bad_feat)]
feat_rest_t = feat_rest_t.rename(columns={'stType': 'target', 
                                          'posX': 'pos_x_t', 
                                          'posY': 'pos_y_t',
                                         'trialN': 'trial_t',
                                         'participantN': 'p_n_t'})
feat_rest_t['pos_x_t'] = np.around(feat_rest_t['pos_x_t'], decimals=2)
feat_rest_t['pos_y_t'] = np.around(feat_rest_t['pos_y_t'], decimals=2)
feat_rest_t['pos_t'] = list(zip(feat_rest_t['pos_x_t'], feat_rest_t['pos_y_t']))
feat_rest_t = feat_rest_t.loc[:, feat_rest_t.columns.intersection(['target',
                                                                   'pos_t',
                                                                   'trial_t',
                                                                  'p_n_t'])]
feat_rest_t = feat_rest_t.reset_index(drop=True)

In [14]:
#numbers of trials in list
trials = feat_rest_t['trial_t'].unique()
trials = trials.astype(np.int32).tolist()

In [15]:
participant = feat_rest_t['p_n_t'].unique()
participant = participant.astype(np.int32).tolist()

In [16]:
dfs_tar_f = dict(tuple(feat_rest_t.groupby('target')))
feat_t1 = dfs_tar_f['target1']
feat_t2 = dfs_tar_f['target2']

In [17]:
t1 = dict(tuple(feat_t1.groupby('trial_t')))
t2 = dict(tuple(feat_t2.groupby('trial_t')))

In [18]:
def closest_node(node, nodes):
    indexArr = np.argwhere(nodes == node)
    nodes = np.delete(nodes,indexArr, 0)
    closest_index = distance.cdist([node], nodes, 'euclidean').argmin()
    return nodes[closest_index]

In [19]:
# %%pixie_debugger 
closest_same = []
for n in range(0, len(participant)):
    part = participant[n]
    for i in range(0, len(trials)):
        n_tr = trials[i]
        t1_arr = np.array(t1.get(trials[i])['pos_t'].values.tolist())
        t2_arr = np.array(t2.get(trials[i])['pos_t'].values.tolist())
        for j in range(0, len(feat_r)):
            if part == feat_r[j][0]:
                if n_tr == feat_r[j][1]:
                    if feat_r[j][2] == 'target1':
                        closest_t1 = closest_node(feat_r[j][3], t1_arr)                          
                        closest_same.append(closest_t1.tolist())         
                    else:
                        closest_t2 = closest_node(feat_r[j][3], t2_arr)                         
                        closest_same.append(closest_t2.tolist())
                else:
                    i += 1
            else:
                n += 1
               

In [20]:
resp_feat['closest_same_target'] = closest_same

In [21]:
closest_other = []
for n in range(0, len(participant)):
    part = participant[n]
    for i in range(0, len(trials)):
        n_tr = trials[i]
        t1_arr = np.array(t1.get(trials[i])['pos_t'].values.tolist())
        t2_arr = np.array(t2.get(trials[i])['pos_t'].values.tolist())
        for j in range(0, len(feat_r)):
            if part == feat_r[j][0]:
                if n_tr == feat_r[j][1]:
                    if feat_r[j][2] == 'target2':
                        closest_t1 = closest_node(feat_r[j][3], t1_arr)                          
                        closest_other.append(closest_t1.tolist())         
                    else:
                        closest_t2 = closest_node(feat_r[j][3], t2_arr)                         
                        closest_other.append(closest_t2.tolist())
                else:
                    i += 1
            else:
                n += 1

In [22]:
resp_feat['closest_other_taget'] = closest_other

In [23]:
feat_rest_d = feat_distr[~feat_distr['trialN']
          .isin(bad_feat)]
feat_rest_d = feat_rest_d.rename(columns={'stType': 'distractor', 
                                          'posX': 'pos_x_d', 
                                          'posY': 'pos_y_d',
                                         'trialN': 'trial_d',
                                         'participantN': 'p_n_d'})
feat_rest_d['pos_x_d'] = np.around(feat_rest_d['pos_x_d'], decimals=2)
feat_rest_d['pos_y_d'] = np.around(feat_rest_d['pos_y_d'], decimals=2)
feat_rest_d['pos_d'] = list(zip(feat_rest_d['pos_x_d'], feat_rest_d['pos_y_d']))
feat_rest_d = feat_rest_d.loc[:, feat_rest_d.columns.intersection(['distractor',
                                                                   'pos_d',
                                                                   'trial_d',
                                                                  'p_n_d'])]
feat_rest_d = feat_rest_d.reset_index(drop=True)

In [24]:
dfs_d_f = dict(tuple(feat_rest_d.groupby('distractor')))
feat_d1 = dfs_d_f['distractor1']
feat_d2 = dfs_d_f['distractor2']
d1 = dict(tuple(feat_d1.groupby('trial_d')))
d2 = dict(tuple(feat_d2.groupby('trial_d')))

In [25]:
# closest blue dot
closest_same_d = []
for n in range(0, len(participant)):
    part = participant[n]
    for i in range(0, len(trials)):
        n_tr = trials[i]
        d1_arr = np.array(d1.get(trials[i])['pos_d'].values.tolist())
        for j in range(0, len(feat_r)):
            if part == feat_r[j][0]:
                if n_tr == feat_r[j][1]:
                    closest_d1 = closest_node(feat_r[j][3], d1_arr)                          
                    closest_same_d.append(closest_d1.tolist())         
                else:
                    i += 1
            else:
                n += 1

In [26]:
resp_feat['closest_blue_dot'] = closest_same_d

In [27]:
# closest yellow dot
closest_d2 = []
for n in range(0, len(participant)):
    part = participant[n]
    for i in range(0, len(trials)):
        n_tr = trials[i]
        d1_arr = np.array(d2.get(trials[i])['pos_d'].values.tolist())
        for j in range(0, len(feat_r)):
            if part == feat_r[j][0]:
                if n_tr == feat_r[j][1]:
                    c_d1 = closest_node(feat_r[j][3], d1_arr)                          
                    closest_d2.append(c_d1.tolist())         
                else:
                    i += 1
            else:
                n += 1

In [28]:
resp_feat['closest_yellow_dot'] = closest_d2

# Grouping_feat

In [29]:
def close_or_not(x1, x2, y1, y2, lst1, lst2):
    count = 0
    for i in range(0, len(lst1)):
        if x2 < lst1[i] < x1 and y2 < lst2[i] < y1:
            count += 1
    return count

In [30]:
def close_or_not_new(x, y, lst1, lst2):
    count = 0
    x1 = x + 108.6
    x2 = x - 108.6
    y1 = y + 106.5
    y2 = y - 106.5
    for i in range(0, len(lst1)):
        if x2 < lst1[i] < x1 and y2 < lst2[i] < y1:
            count += 1
    return count

## Grouping around selected target

In [31]:
feat_r = resp_feat_c.values.tolist()

In [32]:
feat_t1[['pos_x', 'pos_y']] = pd.DataFrame(feat_t1['pos_t'].tolist(), index=feat_t1.index) 
feat_t2[['pos_x', 'pos_y']] = pd.DataFrame(feat_t2['pos_t'].tolist(), index=feat_t1.index)

In [33]:
t1 = dict(tuple(feat_t1.groupby('trial_t')))
t2 = dict(tuple(feat_t2.groupby('trial_t')))

In [34]:
crowding = []
for n in range(0, len(participant)):
    part = participant[n]
    for i in range(0, len(trials)):
        n_tr = trials[i]
        t1_arr_x = t1.get(trials[i])['pos_x'].values.tolist()
        t1_arr_y = t1.get(trials[i])['pos_y'].values.tolist()
        t2_arr_x = t2.get(trials[i])['pos_x'].values.tolist()
        t2_arr_y = t2.get(trials[i])['pos_y'].values.tolist()
        
        for j in range(0, len(feat_r)):
            if part == feat_r[j][0]:
                if n_tr == feat_r[j][1]:
                    if feat_r[j][2] == 'target1':
                        closest_t1 = close_or_not(feat_r[j][6],
                                                  feat_r[j][7],
                                                  feat_r[j][8],
                                                  feat_r[j][9],
                                                  t1_arr_x, t1_arr_y)                          
                        crowding.append(closest_t1)         
                    else:
                        closest_t2 = close_or_not(feat_r[j][6],
                                                  feat_r[j][7],
                                                  feat_r[j][8],
                                                  feat_r[j][9],
                                                  t2_arr_x, t2_arr_y)                          
                        crowding.append(closest_t2)
                else:
                    i += 1
            else:
                n += 1

In [35]:
resp_feat['group_selected'] = crowding

In [36]:
resp_feat

,participantN,age,gender,trialN,task,condition,FeatConj,setSize,proportion,timelimit,...,touchN,error,switches,changeScreen,resp_pos,closest_same_target,closest_other_taget,closest_blue_dot,closest_yellow_dot,group_selected
0,133,0,NaN,0,0,r/g,0,80,50,0.0,...,1,0,2,0,"[124.48, 129.57]","[231.49, 124.62]","[377.55, 205.09]","[122.84, 171.71]","[208.04, 190.76]",2
1,133,0,NaN,0,0,r/g,0,80,50,0.0,...,2,0,0,0,"[394.85, 97.54]","[471.95, 174.93]","[453.62, 124.99]","[368.26, 290.88]","[283.34, 199.65]",2
2,133,0,NaN,0,0,r/g,0,80,50,0.0,...,3,0,0,0,"[231.49, 124.62]","[277.45, 125.85]","[377.55, 205.09]","[122.84, 171.71]","[208.04, 190.76]",3
3,133,0,NaN,0,0,r/g,0,80,50,0.0,...,4,0,0,0,"[471.95, 174.93]","[475.2, 283.61]","[453.62, 124.99]","[368.26, 290.88]","[477.42, 345.82]",4
4,133,0,NaN,0,0,r/g,0,80,50,0.0,...,7,0,1,0,"[546.68, 184.74]","[453.62, 124.99]","[471.95, 174.93]","[632.86, 285.43]","[655.81, 186.53]",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,133,0,NaN,11,0,r/g,0,80,50,0.0,...,41,0,1,0,"[721.66, 335.44]","[635.09, 414.9]","[733.64, 271.54]","[634.7, 333.35]","[795.52, 261.99]",0
396,133,0,NaN,11,0,r/g,0,80,50,0.0,...,43,0,0,0,"[917.05, 410.86]","[788.75, 553.83]","[749.13, 400.0]","[904.86, 353.12]","[826.02, 432.72]",0
397,133,0,NaN,11,0,r/g,0,80,50,0.0,...,44,0,0,0,"[788.75, 553.83]","[704.2, 598.01]","[750.67, 661.48]","[799.22, 494.15]","[723.19, 510.42]",0
398,133,0,NaN,11,0,r/g,0,80,50,0.0,...,45,0,1,0,"[890.71, 660.99]","[647.96, 659.7]","[788.75, 553.83]","[872.5, 595.98]","[723.19, 510.42]",1


## Grouping closest same

In [37]:
feat_cl = resp_feat.loc[:, resp_feat.columns.intersection(['participantN',
                                                                   'trialN',
                                                           'stType',
                                                                   'closest_same_target',
                                                                  'closest_other_taget'])]

In [38]:
feat_cl[['same_x', 'same_y']] = pd.DataFrame(feat_cl['closest_same_target'].tolist(), index=feat_cl.index) 
feat_cl[['switch_x', 'switch_y']] = pd.DataFrame(feat_cl['closest_other_taget'].tolist(), index=feat_cl.index)

In [39]:
feat_clos = feat_cl.values.tolist()

In [40]:
crowding_same = []
for n in range(0, len(participant)):
    part = participant[n]
    for i in range(0, len(trials)):
        n_tr = trials[i]
        t1_arr_x = t1.get(trials[i])['pos_x'].values.tolist()
        t1_arr_y = t1.get(trials[i])['pos_y'].values.tolist()
        t2_arr_x = t2.get(trials[i])['pos_x'].values.tolist()
        t2_arr_y = t2.get(trials[i])['pos_y'].values.tolist()
        
        for j in range(0, len(feat_clos)):
            if part == feat_clos[j][0]:
                if n_tr == feat_clos[j][1]:
                    if feat_clos[j][2] == 'target1':
                        closest_t1 = close_or_not_new(feat_clos[j][5],
                                                      feat_clos[j][6],
                                                  t1_arr_x, t1_arr_y)                          
                        crowding_same.append(closest_t1)         
                    else:
                        closest_t2 = close_or_not_new(feat_clos[j][5],
                                                  feat_clos[j][6],
                                                  t2_arr_x, t2_arr_y)                          
                        crowding_same.append(closest_t2)
                else:
                    i += 1
            else:
                n += 1

In [41]:
resp_feat['group_same'] = crowding_same

## Grouping closest switch

In [42]:
crowding_switch = []
for n in range(0, len(participant)):
    part = participant[n]
    for i in range(0, len(trials)):
        n_tr = trials[i]
        t1_arr_x = t1.get(trials[i])['pos_x'].values.tolist()
        t1_arr_y = t1.get(trials[i])['pos_y'].values.tolist()
        t2_arr_x = t2.get(trials[i])['pos_x'].values.tolist()
        t2_arr_y = t2.get(trials[i])['pos_y'].values.tolist()
        
        for j in range(0, len(feat_clos)):
            if part == feat_clos[j][0]:
                if n_tr == feat_clos[j][1]:
                    if feat_clos[j][2] == 'target1':
                        closest_t1 = close_or_not_new(feat_clos[j][7],
                                                      feat_clos[j][8],
                                                  t1_arr_x, t1_arr_y)                          
                        crowding_switch.append(closest_t1)         
                    else:
                        closest_t2 = close_or_not_new(feat_clos[j][7],
                                                  feat_clos[j][8],
                                                  t2_arr_x, t2_arr_y)                          
                        crowding_switch.append(closest_t2)
                else:
                    i += 1
            else:
                n += 1

In [43]:
resp_feat['group_switch'] = crowding_switch

# Feature strict line

In [44]:
all_feat_sl = all_feat.loc[:, all_feat.columns.intersection(['participantN',
                                                             'trialN',
                                                         'stType',
                                                         'posX',
                                                         'posY'])]

In [45]:
targ = dict(tuple(all_feat_sl.groupby('stType')))
feat_t1 = targ['target1']
feat_t2 = targ['target2']

In [46]:
t1 = dict(tuple(feat_t1.groupby('trialN')))
t2 = dict(tuple(feat_t2.groupby('trialN')))

In [47]:
resp_feat_sl = resp_feat.loc[:, resp_feat.columns.intersection(['participantN',
                                                                'trialN',
                                                                'stType',
                                                                'resp_pos',
                                                                'closest_same_target'])]

In [48]:
resp_feat_sl[['cl_x', 'cl_y']] = pd.DataFrame(resp_feat_sl['closest_same_target']
                                              .tolist(), index=resp_feat_sl.index) 
resp_feat_sl[['r_x', 'r_y']] = pd.DataFrame(resp_feat_sl['resp_pos']
                                              .tolist(), index=resp_feat_sl.index) 

In [49]:
trials = all_feat_sl['trialN'].unique()
trials = trials.astype(np.int32).tolist()

In [50]:
feat_r_sl = resp_feat_sl.values.tolist()

In [51]:
resp_feat_sl = resp_feat.loc[:, resp_feat.columns.intersection(['participantN',
                                                                'trialN',
                                                                'stType',
                                                                'resp_pos',
                                                                'closest_same_target'])]

In [52]:
resp_feat_sl[['cl_x', 'cl_y']] = pd.DataFrame(resp_feat_sl['closest_same_target']
                                              .tolist(), index=resp_feat_sl.index) 
resp_feat_sl[['r_x', 'r_r']] = pd.DataFrame(resp_feat_sl['resp_pos']
                                              .tolist(), index=resp_feat_sl.index) 

In [53]:
trials = resp_feat_sl['trialN'].unique()
trials = trials.astype(np.int32).tolist()

In [54]:
feat_r_sl = resp_feat_sl.values.tolist()

In [55]:
strict_line = []
for n in range(0, len(participant)):
    part = participant[n]
    for i in range(0, len(trials)):
        n_tr = trials[i]
        t1_arr_x = t1.get(trials[i])['posX'].values.tolist()
        t1_arr_y = t1.get(trials[i])['posY'].values.tolist()
        t2_arr_x = t2.get(trials[i])['posX'].values.tolist()
        t2_arr_y = t2.get(trials[i])['posY'].values.tolist()
        
        for j in range(0, len(feat_r_sl)):
            if part == feat_r_sl[j][0]:
                if n_tr == feat_r_sl[j][1]:
                    if feat_clos[j][2] == 'target1':
                        strict = check_area(feat_r_sl[j][7], 
                                            feat_r_sl[j][8],
                                            feat_r_sl[j][5], 
                                            feat_r_sl[j][6],
                                            t2_arr_x, t2_arr_y)
                        strict_line.append(strict)
                    else:
                        strict = check_area(feat_r_sl[j][7], 
                                            feat_r_sl[j][8],
                                            feat_r_sl[j][5], 
                                            feat_r_sl[j][6],
                                            t1_arr_x, t1_arr_y)
                        strict_line.append(strict)
                        
            else:
                i += 1
        else:
            n += 1

In [56]:
can_provide_line = []
for i in range(0, len(strict_line)):
        if 1 in strict_line[i]:
            can_provide_line.append(1)
        else:
            can_provide_line.append(0)
    

In [57]:
resp_feat['strict_line'] = can_provide_line

In [58]:
resp_feat['blocked_grouped'] = [1 if x > 0 & y>0 else 0 
                              for x, y in zip(resp_feat['group_same'],resp_feat['strict_line'])]

In [59]:
resp_feat['blocked_grouped'].unique()

array([0], dtype=int64)

## Run length and Ratio between T1 and T2

In [60]:
ratio_feat = []
for n in range(0, len(participant)):
    part = participant[n]
    for i in range(0, len(trials)):
        n_tr = trials[i]
        t1_r = len(t1.get(trials[0])['stType'].values.tolist())
        t2_r = len(t2.get(trials[0])['stType'].values.tolist())
        count_t1 = []
        count_t2 = []
        for j in range(0, len(feat_r)):
            if part == feat_r[j][0]:
                if n_tr == feat_r[j][1]:
                    if feat_r[j][2] == 'target1' or feat_r[j][2] == ' target1 ':
                        count_t1.append(1)
                        a = t1_r - len(count_t1)
                        b = t2_r - len(count_t2)
                        #print(len(count_t1))
                        rat_t1 = math.log10((a + 1) / (b + 1))
                        ratio_feat.append(rat_t1)
                    elif feat_r[j][2] == 'target2' or feat_r[j][2] == ' target2 ':
                        a = t1_r - len(count_t1)
                        b = t2_r - len(count_t2)
                        count_t2.append(1)
                        rat_t2 = math.log10((b + 1) / (a + 1))
                        ratio_feat.append(rat_t2)   

In [61]:
resp_feat['ratio'] = ratio_feat

In [62]:
resp_feat

,participantN,age,gender,trialN,task,condition,FeatConj,setSize,proportion,timelimit,...,closest_same_target,closest_other_taget,closest_blue_dot,closest_yellow_dot,group_selected,group_same,group_switch,strict_line,blocked_grouped,ratio
0,133,0,NaN,0,0,r/g,0,80,50,0.0,...,"[231.49, 124.62]","[377.55, 205.09]","[122.84, 171.71]","[208.04, 190.76]",2,3,3,0,0,-0.021189
1,133,0,NaN,0,0,r/g,0,80,50,0.0,...,"[471.95, 174.93]","[453.62, 124.99]","[368.26, 290.88]","[283.34, 199.65]",2,4,3,0,0,-0.043466
2,133,0,NaN,0,0,r/g,0,80,50,0.0,...,"[277.45, 125.85]","[377.55, 205.09]","[122.84, 171.71]","[208.04, 190.76]",3,2,3,0,0,-0.066947
3,133,0,NaN,0,0,r/g,0,80,50,0.0,...,"[475.2, 283.61]","[453.62, 124.99]","[368.26, 290.88]","[477.42, 345.82]",4,2,3,0,0,-0.091770
4,133,0,NaN,0,0,r/g,0,80,50,0.0,...,"[453.62, 124.99]","[471.95, 174.93]","[632.86, 285.43]","[655.81, 186.53]",0,0,0,0,0,0.091770
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,133,0,NaN,11,0,r/g,0,80,50,0.0,...,"[635.09, 414.9]","[733.64, 271.54]","[634.7, 333.35]","[795.52, 261.99]",0,0,0,0,0,0.124939
396,133,0,NaN,11,0,r/g,0,80,50,0.0,...,"[788.75, 553.83]","[749.13, 400.0]","[904.86, 353.12]","[826.02, 432.72]",0,0,0,0,0,0.000000
397,133,0,NaN,11,0,r/g,0,80,50,0.0,...,"[704.2, 598.01]","[750.67, 661.48]","[799.22, 494.15]","[723.19, 510.42]",0,0,0,0,0,-0.176091
398,133,0,NaN,11,0,r/g,0,80,50,0.0,...,"[647.96, 659.7]","[788.75, 553.83]","[872.5, 595.98]","[723.19, 510.42]",1,2,0,0,0,0.301030


# Conjunction task

In [63]:
resp_conj = responses[responses["FeatConj"] == 1]
bad_conj = resp_conj.loc[(resp_conj['stType'] == ' distractor2 ') | (resp_conj['stType'] == ' distractor1 ') | (resp_conj['stType'] == 'distractor2') | (resp_conj['stType'] == 'distractor1'), 
                         'trialN'].tolist()
resp_conj = resp_conj[~resp_conj['trialN']
          .isin(bad_conj)]
resp_conj = resp_conj.reset_index(drop=True)

In [64]:
resp_conj1 = resp_conj.rename(columns={'stType': 'type', 
                                          'stPosX': 'pos_x_r', 
                                          'stPosY': 'pos_y_r',
                                         'trialN': 'trial_r',
                                      'participantN': 'p_n'})
resp_conj1['pos_x_r'] = np.around(resp_conj1['pos_x_r'], decimals=2)
resp_conj1['pos_y_r'] = np.around(resp_conj1['pos_y_r'], decimals=2)
resp_conj['resp_pos'] = resp_conj1.apply(lambda x: list([x['pos_x_r'], x['pos_y_r']]) , axis=1)
resp_conj1['pos_r'] = list(zip(resp_conj1['pos_x_r'], resp_conj1['pos_y_r']))

resp_conj_c = resp_conj1.loc[:, resp_conj1.columns.intersection(['type', 
                                                             'pos_r',
                                                             'trial_r',
                                                               'p_n',
                                                               'pos_x_r',
                                                               'pos_y_r'])]

resp_conj1 = resp_conj1.loc[:, resp_conj1.columns.intersection(['type', 
                                                             'pos_r',
                                                             'trial_r',
                                                               'p_n'])]

resp_conj1 = resp_conj1.reset_index(drop=True)
resp_conj1.type = resp_conj1.type.str.strip()

### Add x1,x2,y1,y2

In [65]:
resp_conj_c['x1'] = resp_conj_c['pos_x_r'].apply(lambda x: x + 108.6)
resp_conj_c['x2'] = resp_conj_c['pos_x_r'].apply(lambda x: x - 108.6)
resp_conj_c['y1'] = resp_conj_c['pos_y_r'].apply(lambda x: x + 106.5)
resp_conj_c['y2'] = resp_conj_c['pos_y_r'].apply(lambda x: x - 106.5)

### Distances

In [66]:
conj_r = resp_conj1.values.tolist()
## [prtN, trialN, target_type, pos]

In [67]:
#only useful df of all stimuli for conjunction task
all_conj = all_stim[all_stim["FeatConj"] == 1]
conj_target = all_conj[(all_conj['stType'] == 'target1') |
                       (all_conj['stType'] == 'target2')]
conj_distr = all_conj[(all_conj['stType'] == 'distractor1') |
                      (all_conj['stType'] == 'distractor2')]

In [68]:
#useful targets for conjunction
conj_rest_t = conj_target[~conj_target['trialN']
          .isin(bad_conj)]
conj_rest_t = conj_rest_t.rename(columns={'stType': 'target', 
                                          'posX': 'pos_x_t', 
                                          'posY': 'pos_y_t',
                                         'trialN': 'trial_t',
                                         'participantN': 'p_n_t'})
conj_rest_t['pos_x_t'] = np.around(conj_rest_t['pos_x_t'], decimals=2)
conj_rest_t['pos_y_t'] = np.around(conj_rest_t['pos_y_t'], decimals=2)
conj_rest_t['pos_t'] = list(zip(conj_rest_t['pos_x_t'], conj_rest_t['pos_y_t']))
conj_rest_t = conj_rest_t.loc[:, conj_rest_t.columns.intersection(['target',
                                                                   'pos_t',
                                                                   'trial_t',
                                                                  'p_n_t'])]
conj_rest_t = conj_rest_t.reset_index(drop=True)

In [69]:
trials = conj_rest_t['trial_t'].unique()
trials = trials.astype(np.int32).tolist()

In [70]:
participant = conj_rest_t['p_n_t'].unique()
participant = participant.astype(np.int32).tolist()

In [71]:
dfs_tar_c = dict(tuple(conj_rest_t.groupby('target')))
conj_t1 = dfs_tar_c['target1']
conj_t2 = dfs_tar_c['target2']

In [72]:
t1 = dict(tuple(conj_t1.groupby('trial_t')))
t2 = dict(tuple(conj_t2.groupby('trial_t')))

In [73]:
closest_same = []
for n in range(0, len(participant)):
    part = participant[n]
    for i in range(0, len(trials)):
        n_tr = trials[i]
        t1_arr = np.array(t1.get(trials[i])['pos_t'].values.tolist())
        t2_arr = np.array(t2.get(trials[i])['pos_t'].values.tolist())
        for j in range(0, len(conj_r)):
            if part == conj_r[j][0]:
                if n_tr == conj_r[j][1]:
                    if conj_r[j][2] == 'target1':
                        closest_t1 = closest_node(conj_r[j][3], t1_arr)                          
                        closest_same.append(closest_t1.tolist())         
                    else:
                        closest_t2 = closest_node(conj_r[j][3], t2_arr)                         
                        closest_same.append(closest_t2.tolist())
                else:
                    i += 1
            else:
                n += 1

In [74]:
resp_conj['closest_same_target'] = closest_same

In [75]:
closest_other = []
for n in range(0, len(participant)):
    part = participant[n]
    for i in range(0, len(trials)):
        n_tr = trials[i]
        t1_arr = np.array(t1.get(trials[i])['pos_t'].values.tolist())
        t2_arr = np.array(t2.get(trials[i])['pos_t'].values.tolist())
        for j in range(0, len(conj_r)):
            if part == conj_r[j][0]:
                if n_tr == conj_r[j][1]:
                    if conj_r[j][2] == 'target2':
                        closest_t1 = closest_node(conj_r[j][3], t1_arr)                          
                        closest_other.append(closest_t1.tolist())         
                    else:
                        closest_t2 = closest_node(conj_r[j][3], t2_arr)                         
                        closest_other.append(closest_t2.tolist())
                else:
                    i += 1
            else:
                n += 1

In [76]:
resp_conj['closest_other_target'] = closest_other

In [77]:
#useful distractors for conjunction
conj_rest_d = conj_distr[~conj_distr['trialN']
          .isin(bad_conj)]
conj_rest_d = conj_rest_d.rename(columns={'stType': 'distractor', 
                                          'posX': 'pos_x_d', 
                                          'posY': 'pos_y_d',
                                         'trialN': 'trial_d',
                                         'participantN': 'p_n_d'})
conj_rest_d['pos_x_d'] = np.around(conj_rest_d['pos_x_d'], decimals=2)
conj_rest_d['pos_y_d'] = np.around(conj_rest_d['pos_y_d'], decimals=2)
conj_rest_d['pos_d'] = list(zip(conj_rest_d['pos_x_d'], conj_rest_d['pos_y_d']))
conj_rest_d = conj_rest_d.loc[:, conj_rest_d.columns.intersection(['distractor',
                                                                   'pos_d',
                                                                   'trial_d',
                                                                  'p_n_d'])]
conj_rest_d = conj_rest_d.reset_index(drop=True)

In [78]:
dfs_d_c = dict(tuple(conj_rest_d.groupby('distractor')))
conj_d1 = dfs_d_c['distractor1']
conj_d2 = dfs_d_c['distractor2']
d1 = dict(tuple(conj_d1.groupby('trial_d')))
d2 = dict(tuple(conj_d2.groupby('trial_d')))

In [79]:
same_color = []
for n in range(0, len(participant)):
    part = participant[n]
    for i in range(0, len(trials)):
        n_tr = trials[i]
        t1_arr = np.array(d1.get(trials[i])['pos_d'].values.tolist())
        t2_arr = np.array(d2.get(trials[i])['pos_d'].values.tolist())
        for j in range(0, len(conj_r)):
            if part == conj_r[j][0]:
                if n_tr == conj_r[j][1]:
                    if conj_r[j][2] == 'target1':
                        closest_d1 = closest_node(conj_r[j][3], t1_arr)                          
                        same_color.append(closest_d1.tolist())         
                    else:
                        closest_d2 = closest_node(conj_r[j][3], t2_arr)                         
                        same_color.append(closest_d2.tolist())
                else:
                    i += 1
            else:
                n += 1

In [80]:
resp_conj['closest_dist_same_color'] = same_color

In [81]:
same_shape = []
for n in range(0, len(participant)):
    part = participant[n]
    for i in range(0, len(trials)):
        n_tr = trials[i]
        t1_arr = np.array(d1.get(trials[i])['pos_d'].values.tolist())
        t2_arr = np.array(d2.get(trials[i])['pos_d'].values.tolist())
        for j in range(0, len(conj_r)):
            if part == conj_r[j][0]:
                if n_tr == conj_r[j][1]:
                    if conj_r[j][2] == 'target2':
                        closest_t1 = closest_node(conj_r[j][3], t1_arr)                          
                        same_shape.append(closest_t1.tolist())         
                    else:
                        closest_t2 = closest_node(conj_r[j][3], t2_arr)                         
                        same_shape.append(closest_t2.tolist())
                else:
                    i += 1
            else:
                n += 1

In [82]:
resp_conj['closest_dist_same_shape'] = same_shape

# Grouping_conj

In [83]:
conj_r = resp_conj_c.values.tolist()

In [84]:
conj_t1[['pos_x', 'pos_y']] = pd.DataFrame(conj_t1['pos_t'].tolist(), index=conj_t1.index) 
conj_t2[['pos_x', 'pos_y']] = pd.DataFrame(conj_t2['pos_t'].tolist(), index=conj_t1.index)

In [85]:
t1 = dict(tuple(conj_t1.groupby('trial_t')))
t2 = dict(tuple(conj_t2.groupby('trial_t')))

In [86]:
crowding = []
for n in range(0, len(participant)):
    part = participant[n]
    for i in range(0, len(trials)):
        n_tr = trials[i]
        t1_arr_x = t1.get(trials[i])['pos_x'].values.tolist()
        t1_arr_y = t1.get(trials[i])['pos_y'].values.tolist()
        t2_arr_x = t2.get(trials[i])['pos_x'].values.tolist()
        t2_arr_y = t2.get(trials[i])['pos_y'].values.tolist()
        
        for j in range(0, len(conj_r)):
            if part == conj_r[j][0]:
                if n_tr == conj_r[j][1]:
                    if conj_r[j][2] == 'target1':
                        closest_t1 = close_or_not(conj_r[j][6],
                                                  conj_r[j][7],
                                                  conj_r[j][8],
                                                  conj_r[j][9],
                                                  t1_arr_x, t1_arr_y)                          
                        crowding.append(closest_t1)         
                    else:
                        closest_t2 = close_or_not(conj_r[j][6],
                                                  conj_r[j][7],
                                                  conj_r[j][8],
                                                  conj_r[j][9],
                                                  t2_arr_x, t2_arr_y)                          
                        crowding.append(closest_t2)
                else:
                    i += 1
            else:
                n += 1

In [87]:
resp_conj['grouping_selected'] = crowding

## Crowding closest same

In [88]:
conj_cl = resp_conj.loc[:, resp_conj.columns.intersection(['participantN',
                                                                   'trialN',
                                                           'stType',
                                                                   'closest_same_target',
                                                                  'closest_other_target'])]

In [89]:
conj_cl[['same_x', 'same_y']] = pd.DataFrame(conj_cl['closest_same_target'].tolist(), index=conj_cl.index) 
conj_cl[['switch_x', 'switch_y']] = pd.DataFrame(conj_cl['closest_other_target'].tolist(), index=conj_cl.index)

In [90]:
conj_clos = conj_cl.values.tolist()

In [91]:
crowding_same = []
for n in range(0, len(participant)):
    part = participant[n]
    for i in range(0, len(trials)):
        n_tr = trials[i]
        t1_arr_x = t1.get(trials[i])['pos_x'].values.tolist()
        t1_arr_y = t1.get(trials[i])['pos_y'].values.tolist()
        t2_arr_x = t2.get(trials[i])['pos_x'].values.tolist()
        t2_arr_y = t2.get(trials[i])['pos_y'].values.tolist()
        
        for j in range(0, len(conj_clos)):
            if part == conj_clos[j][0]:
                if n_tr == conj_clos[j][1]:
                    if conj_clos[j][2] == 'target1':
                        closest_t1 = close_or_not_new(conj_clos[j][5],
                                                      conj_clos[j][6],
                                                  t1_arr_x, t1_arr_y)                          
                        crowding_same.append(closest_t1)         
                    else:
                        closest_t2 = close_or_not_new(conj_clos[j][5],
                                                  conj_clos[j][6],
                                                  t2_arr_x, t2_arr_y)                          
                        crowding_same.append(closest_t2)
                else:
                    i += 1
            else:
                n += 1

In [92]:
resp_conj['grouping_same'] = crowding_same

## Crowding closest switch

In [93]:
crowding_switch = []
for n in range(0, len(participant)):
    part = participant[n]
    for i in range(0, len(trials)):
        n_tr = trials[i]
        t1_arr_x = t1.get(trials[i])['pos_x'].values.tolist()
        t1_arr_y = t1.get(trials[i])['pos_y'].values.tolist()
        t2_arr_x = t2.get(trials[i])['pos_x'].values.tolist()
        t2_arr_y = t2.get(trials[i])['pos_y'].values.tolist()
        
        for j in range(0, len(conj_clos)):
            if part == conj_clos[j][0]:
                if n_tr == conj_clos[j][1]:
                    if conj_clos[j][2] == 'target1':
                        closest_t1 = close_or_not_new(conj_clos[j][7],
                                                      conj_clos[j][8],
                                                  t1_arr_x, t1_arr_y)                          
                        crowding_switch.append(closest_t1)         
                    else:
                        closest_t2 = close_or_not_new(conj_clos[j][7],
                                                  conj_clos[j][8],
                                                  t2_arr_x, t2_arr_y)                          
                        crowding_switch.append(closest_t2)
                else:
                    i += 1
            else:
                n += 1

In [94]:
resp_conj['grouping_switch'] = crowding_switch

## Strict line conj

In [95]:
all_conj_sl = all_conj.loc[:, all_conj.columns.intersection(['participantN',
                                                             'trialN',
                                                         'stType',
                                                         'posX',
                                                         'posY'])]

In [96]:
targ = dict(tuple(all_conj_sl.groupby('stType')))
conj_t1 = targ['target1']
conj_t2 = targ['target2']

In [97]:
t1 = dict(tuple(conj_t1.groupby('trialN')))
t2 = dict(tuple(conj_t2.groupby('trialN')))

In [98]:
resp_conj_sl = resp_conj.loc[:, resp_conj.columns.intersection(['participantN',
                                                                'trialN',
                                                                'stType',
                                                                'resp_pos',
                                                                'closest_same_target'])]

In [99]:
resp_conj_sl[['cl_x', 'cl_y']] = pd.DataFrame(resp_conj_sl['closest_same_target']
                                              .tolist(), index=resp_conj_sl.index) 
resp_conj_sl[['r_x', 'r_y']] = pd.DataFrame(resp_conj_sl['resp_pos']
                                              .tolist(), index=resp_conj_sl.index) 

In [100]:
trials = all_conj_sl['trialN'].unique()
trials = trials.astype(np.int32).tolist()

In [101]:
conj_r_sl = resp_conj_sl.values.tolist()

In [102]:
strict_line = []
for n in range(0, len(participant)):
    part = participant[n]
    for i in range(0, len(trials)):
        n_tr = trials[i]
        t1_arr_x = t1.get(trials[i])['posX'].values.tolist()
        t1_arr_y = t1.get(trials[i])['posY'].values.tolist()
        t2_arr_x = t2.get(trials[i])['posX'].values.tolist()
        t2_arr_y = t2.get(trials[i])['posY'].values.tolist()
        
        for j in range(0, len(conj_r_sl)):
            if part == conj_r_sl[j][0]:
                if n_tr == conj_r_sl[j][1]:
                    if conj_clos[j][2] == 'target1':
                        strict = check_area(conj_r_sl[j][7], 
                                            conj_r_sl[j][8],
                                            conj_r_sl[j][5], 
                                            conj_r_sl[j][6],
                                            t2_arr_x, t2_arr_y)
                        strict_line.append(strict)
                    else:
                        strict = check_area(conj_r_sl[j][7], 
                                            conj_r_sl[j][8],
                                            conj_r_sl[j][5], 
                                            conj_r_sl[j][6],
                                            t1_arr_x, t1_arr_y)
                        strict_line.append(strict)
                        
            else:
                i += 1
        else:
            n += 1

In [103]:
can_provide_line = []
for i in range(0, len(strict_line)):
    if 1 in strict_line[i]:
        can_provide_line.append(1)
    else:
        can_provide_line.append(0)
    

In [104]:
resp_conj['strict_line'] = can_provide_line

In [105]:
resp_conj['blocked_grouped'] = [1 if x > 0 & y > 0 else 0 
                              for x, y in zip(resp_conj['grouping_same'],resp_conj['strict_line'])]

In [106]:
resp_conj['blocked_grouped'].unique()

array([0], dtype=int64)

In [107]:
resp_conj['strict_line'].unique()

array([0, 1], dtype=int64)

## Ratio conj

In [108]:
ratio_conj = []
for n in range(0, len(participant)):
    part = participant[n]
    for i in range(0, len(trials)):
        n_tr = trials[i]
        t1_r = len(t1.get(trials[0])['stType'].values.tolist())
        t2_r = len(t2.get(trials[0])['stType'].values.tolist())
        count_t1 = []
        count_t2 = []
        for j in range(0, len(conj_r)):
            if n_tr == conj_r[j][1]:
                if conj_r[j][2] == 'target1' or conj_r[j][2] == ' target1 ':
                    count_t1.append(1)
                    rat_t1 = math.log10(((t1_r - len(count_t1)) + 1)/ ((t2_r - len(count_t2)) + 1))
                    ratio_conj.append(rat_t1)
                elif conj_r[j][2] == 'target2' or conj_r[j][2] == ' target2 ':
                    count_t2.append(1)
                    rat_t2 = math.log10(((t2_r - len(count_t2)) + 1)/ ((t1_r - len(count_t1)) + 1))
                    ratio_conj.append(rat_t2)
                    
                

In [109]:
resp_conj['ratio'] = ratio_conj

# Make csv

In [110]:
resp_feat.head()

,participantN,age,gender,trialN,task,condition,FeatConj,setSize,proportion,timelimit,...,closest_same_target,closest_other_taget,closest_blue_dot,closest_yellow_dot,group_selected,group_same,group_switch,strict_line,blocked_grouped,ratio
0,133,0,NaN,0,0,r/g,0,80,50,0.0,...,"[231.49, 124.62]","[377.55, 205.09]","[122.84, 171.71]","[208.04, 190.76]",2,3,3,0,0,-0.021189
1,133,0,NaN,0,0,r/g,0,80,50,0.0,...,"[471.95, 174.93]","[453.62, 124.99]","[368.26, 290.88]","[283.34, 199.65]",2,4,3,0,0,-0.043466
2,133,0,NaN,0,0,r/g,0,80,50,0.0,...,"[277.45, 125.85]","[377.55, 205.09]","[122.84, 171.71]","[208.04, 190.76]",3,2,3,0,0,-0.066947
3,133,0,NaN,0,0,r/g,0,80,50,0.0,...,"[475.2, 283.61]","[453.62, 124.99]","[368.26, 290.88]","[477.42, 345.82]",4,2,3,0,0,-0.091770
4,133,0,NaN,0,0,r/g,0,80,50,0.0,...,"[453.62, 124.99]","[471.95, 174.93]","[632.86, 285.43]","[655.81, 186.53]",0,0,0,0,0,0.091770


In [111]:
resp_conj

,participantN,age,gender,trialN,task,condition,FeatConj,setSize,proportion,timelimit,...,closest_same_target,closest_other_target,closest_dist_same_color,closest_dist_same_shape,grouping_selected,grouping_same,grouping_switch,strict_line,blocked_grouped,ratio
0,133,0,NaN,0,0,rd/gs,1,80,50,0.0,...,"[873.13, 191.35]","[805.45, 257.98]","[888.91, 103.81]","[785.08, 108.5]",3,3,3,0,0,-0.021189
1,133,0,NaN,0,0,rd/gs,1,80,50,0.0,...,"[904.58, 259.51]","[805.45, 257.98]","[888.91, 103.81]","[785.08, 108.5]",3,3,3,0,0,-0.043466
2,133,0,NaN,0,0,rd/gs,1,80,50,0.0,...,"[873.13, 191.35]","[805.45, 257.98]","[746.47, 261.52]","[785.08, 108.5]",3,3,3,0,0,-0.066947
3,133,0,NaN,0,0,rd/gs,1,80,50,0.0,...,"[560.42, 190.45]","[559.25, 360.62]","[656.46, 284.68]","[450.92, 290.14]",2,2,1,0,0,-0.091770
4,133,0,NaN,0,0,rd/gs,1,80,50,0.0,...,"[550.57, 289.01]","[533.61, 137.69]","[665.07, 198.42]","[456.61, 214.15]",2,2,1,0,0,-0.118099
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,133,0,NaN,11,0,rd/gs,1,80,50,0.0,...,"[209.15, 499.94]","[297.34, 441.21]","[393.55, 486.93]","[294.81, 579.88]",0,0,0,0,0,0.301030
396,133,0,NaN,11,0,rd/gs,1,80,50,0.0,...,"[230.74, 633.05]","[400.08, 651.14]","[393.55, 486.93]","[294.81, 579.88]",0,0,0,0,0,0.176091
397,133,0,NaN,11,0,rd/gs,1,80,50,0.0,...,"[284.3, 643.15]","[146.9, 554.92]","[393.55, 486.93]","[226.6, 584.88]",0,0,0,0,0,0.000000
398,133,0,NaN,11,0,rd/gs,1,80,50,0.0,...,"[309.17, 511.85]","[469.48, 557.46]","[393.55, 486.93]","[294.81, 579.88]",0,0,0,0,0,-0.301030


In [112]:
resp_feat.to_csv (r'C:\Users\lttso\Desktop\feature_task.csv', index = False,  mode='a', header=False)

In [113]:
resp_conj.to_csv (r'C:\Users\lttso\Desktop\conjunction_task.csv', index = False, mode='a', header=False)